In [5]:
import pandas as pd
import numpy as np
import os
import cv2
import glob
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

In [6]:
#Connect with google drive 
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/gdrive', force_remount=True)

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/gdrive


In [7]:
DIR_TRAIN = "../content/gdrive/MyDrive/birds/train"
DIR_VALID = "../content/gdrive/MyDrive/birds/valid"
DIR_TEST = "../content/gdrive/MyDrive/birds/test"

In [8]:
birds = np.array(list(os.listdir(DIR_TRAIN)))

In [9]:
nr_birds = 20

np.random.shuffle(birds)
birds = birds[:nr_birds]

In [10]:
idx_to_name = {i:x for (i,x) in enumerate(birds)}
name_to_idx = {x:i for (i,x) in enumerate(birds)}
print(idx_to_name)

{0: 'GILDED FLICKER', 1: 'AMERICAN COOT', 2: 'SPLENDID WREN', 3: 'TAKAHE', 4: 'BALTIMORE ORIOLE', 5: 'VENEZUELIAN TROUPIAL', 6: 'MOURNING DOVE', 7: 'LONG-EARED OWL', 8: 'CASPIAN TERN', 9: 'OVENBIRD', 10: 'AMERICAN PIPIT', 11: 'RED FACED CORMORANT', 12: 'ROADRUNNER', 13: 'RED HONEY CREEPER', 14: 'ROSY FACED LOVEBIRD', 15: 'PURPLE FINCH', 16: 'ANNAS HUMMINGBIRD', 17: 'STRIPPED SWALLOW', 18: 'NORTHERN PARULA', 19: 'MANDRIN DUCK'}


In [11]:
def get_data_labels(path, birds, dim):
    data = []
    labels = []

    for bird in birds:
        imgs = [cv2.resize(cv2.imread(img), dim, interpolation=cv2.INTER_AREA) for img in glob.glob(path + "/" + bird + "/*.jpg")]
        for img in imgs:
            data.append(img)
            labels.append(name_to_idx[bird])
    return np.array(data), np.array(labels)

In [12]:
data_train, labels_train = get_data_labels(DIR_TRAIN, idx_to_name.values(), (224,224))
data_test, labels_test = get_data_labels(DIR_TEST, idx_to_name.values(), (224,224))
data_valid, labels_valid = get_data_labels(DIR_VALID, idx_to_name.values(), (224,224))

In [13]:
def normalize(data):
    data = data / 255.0
    data = data.astype('float32')
    return data

def one_hot(labels):
    labels = np.eye(len(np.unique(labels)))[labels]
    return labels

In [14]:
data_train

array([[[[185, 159, 143],
         [184, 158, 141],
         [184, 158, 141],
         ...,
         [185, 156, 135],
         [184, 155, 134],
         [183, 154, 133]],

        [[185, 159, 143],
         [184, 158, 141],
         [185, 160, 140],
         ...,
         [186, 157, 136],
         [185, 156, 135],
         [185, 156, 135]],

        [[185, 159, 142],
         [184, 159, 139],
         [185, 160, 140],
         ...,
         [187, 158, 137],
         [187, 157, 138],
         [186, 156, 137]],

        ...,

        [[177, 153, 133],
         [177, 153, 133],
         [178, 153, 133],
         ...,
         [125, 134, 121],
         [124, 134, 122],
         [122, 134, 122]],

        [[177, 153, 135],
         [177, 153, 135],
         [177, 152, 132],
         ...,
         [127, 136, 123],
         [126, 135, 122],
         [126, 136, 123]],

        [[179, 155, 137],
         [179, 155, 137],
         [179, 154, 134],
         ...,
         [132, 139, 126],
        

In [15]:
labels_train

array([ 0,  0,  0, ..., 19, 19, 19])

In [16]:
data_train = normalize(data_train)
data_test = normalize(data_test)
data_valid = normalize(data_valid)

In [17]:
labels_train = one_hot(labels_train)
labels_test = one_hot(labels_test)
labels_valid = one_hot(labels_valid)

In [18]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.summary()

58892288/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
______________________________________________________________

In [19]:
for layer in base_model.layers:
    layer.trainable = False
 
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [20]:
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
 
# use “get_layer” method to save the last layer of the network
last_layer = base_model.get_layer('block5_pool')
# save the output of the last layer to be the input of the next layer
last_output = last_layer.output
 
# flatten the classifier input which is output of the last layer of VGG16 model
x = Flatten()(last_output)
 
# add our new softmax layer with 3 hidden units
x = Dense(nr_birds, activation='softmax', name='softmax')(x)

In [21]:
# instantiate a new_model using keras’s Model class
new_model = Model(inputs=base_model.input, outputs=x)
 
# print the new_model summary
new_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [22]:
new_model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [23]:
checkpointer = ModelCheckpoint(filepath='birds.model.hdf5', save_best_only=True)

In [24]:
history = new_model.fit(data_train, labels_train, steps_per_epoch=len(data_train),
validation_data=(data_test, labels_test), validation_steps=3, epochs=5, verbose=1, callbacks=[checkpointer])

Epoch 1/5
2869/2869 [==============================] - 69s 13ms/step - loss: 1.0298 - accuracy: 0.7079 - val_loss: 0.3751 - val_accuracy: 0.9200
Epoch 2/5
2869/2869 [==============================] - 26s 9ms/step - loss: 0.1556 - accuracy: 0.9669 - val_loss: 0.2356 - val_accuracy: 0.9400
Epoch 3/5
2869/2869 [==============================] - 27s 9ms/step - loss: 0.0457 - accuracy: 0.9951 - val_loss: 0.2091 - val_accuracy: 0.9400
Epoch 4/5
2869/2869 [==============================] - 27s 9ms/step - loss: 0.0201 - accuracy: 0.9983 - val_loss: 0.1847 - val_accuracy: 0.9500
Epoch 5/5
2869/2869 [==============================] - 27s 10ms/step - loss: 0.0079 - accuracy: 0.9990 - val_loss: 0.1720 - val_accuracy: 0.9300


In [25]:
new_model.save(os.curdir)

INFO:tensorflow:Assets written to: ./assets


In [26]:
new_model.save('content/finalmodel/')

INFO:tensorflow:Assets written to: content/finalmodel/assets


In [27]:
import tensorflow as tf
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model("content/finalmodel/") # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [28]:
new_load_model = tf.keras.models.load_model("content/finalmodel/")

In [55]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
##input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
dir='/content/Roadrunner_DeathValley.jpg'
input_data=cv2.imread(dir).astype("float32")
input_data=cv2.resize(input_data,(224,224))
input_data=np.expand_dims(input_data,0)#2D->3D
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[2.938265e-39 2.938265e-39 2.938265e-39 2.938265e-39 2.938265e-39
  2.938265e-39 2.938265e-39 2.938265e-39 2.938265e-39 2.938265e-39
  2.938265e-39 2.938265e-39 1.000000e+00 2.938265e-39 2.938265e-39
  2.938265e-39 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00]]


In [54]:
input_data=cv2.imread(dir).astype("float32")
type(input_data)

input_data.shape
##input_data=np.expand_dims(input_data,0)#2D->3D
input_data=cv2.resize(input_data,(224,224))
print(input_data.shape)

(224, 224, 3)


In [56]:
x=np.argmax(output_data)
print(x)
## the index

12


In [58]:
idx_to_name[x]

'ROADRUNNER'

In [30]:
output_details

[{'dtype': numpy.float32,
  'index': 50,
  'name': 'StatefulPartitionedCall:0',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([ 1, 20], dtype=int32),
  'shape_signature': array([-1, 20], dtype=int32),
  'sparsity_parameters': {}}]

In [40]:
input_details

[{'dtype': numpy.float32,
  'index': 0,
  'name': 'serving_default_input_1:0',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([  1, 224, 224,   3], dtype=int32),
  'shape_signature': array([ -1, 224, 224,   3], dtype=int32),
  'sparsity_parameters': {}}]